<a href="https://colab.research.google.com/github/Jaesu26/vime/blob/main/examples/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VIME Example

`-` An example to train VIME-Self and VIME-Semi using google colab gpu

## Install VIME

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/vime

/content/drive/MyDrive/Colab Notebooks/vime


In [3]:
!pip install git+https://github.com/Jaesu26/vime.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Jaesu26/vime.git to /tmp/pip-req-build-1c37wlhw
  Running command git clone --filter=blob:none --quiet https://github.com/Jaesu26/vime.git /tmp/pip-req-build-1c37wlhw
  Resolved https://github.com/Jaesu26/vime.git to commit edf3757cf2857973f28332553d85e3797c3ba880
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 11.8 MB/s e

## Prepare MNIST

In [4]:
!pip install easydict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import logging
import os
import random
import warnings

import easydict
import numpy as np
import torch
import torch.nn as nn

warnings.filterwarnings("ignore") 

- Hyperparameters

In [6]:
args_self = easydict.EasyDict({
    "weights_dirpath": "./vimeself_weights",
    "max_epochs": 50,
    "batch_size": 512,
    "train_size": 0.9,
    "learning_rate": 1e-2, 
    "p_masking": 0.3,
    "alpha": 2.0,
    "log_interval": 5,
    "seed": 26,
})
args_semi = easydict.EasyDict({
    "weights_dirpath": "./vimesemi_weights",
    "num_classes": 10,
    "supervised_criterion": nn.CrossEntropyLoss(),
    "max_epochs": 100,
    "labeled_batch_size": 64,
    "unlabeled_batch_size": 512,
    "train_size": 0.9,
    "learning_rate": 1e-3, 
    "p_masking": 0.3,
    "K": 3,
    "beta": 1.0,
    "log_interval": 5,
    "seed": 26,
})

In [7]:
def create_folder(path: str) -> None:
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError as error:
        print(error)

In [8]:
create_folder(args_self.weights_dirpath)
create_folder(args_semi.weights_dirpath)

- Load data

In [9]:
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [10]:
mnist = fetch_openml("mnist_784")

In [11]:
data = mnist.data.values
target = mnist.target.astype(int).values

In [12]:
data /= 255.0

In [13]:
data.shape

(70000, 784)

- Split data

In [14]:
num_labeled_data_used = 1000
unlabeled_data_rate = 0.9
seed = 26
test_size = 1 / 7

In [15]:
X, X_test, y, y_test = train_test_split(data, target, test_size=test_size, random_state=seed, stratify=target)

In [16]:
X_labeled, X_unlabeled, y, _ = train_test_split(X, y, test_size=unlabeled_data_rate, random_state=seed, stratify=y)

In [17]:
X_labeled = X_labeled[:num_labeled_data_used]
y = y[:num_labeled_data_used]

## VIME

In [18]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from vime.datamodules import VIMESelfDataModule, VIMESemiDataModule
from vime.lightningmodules import VIMESelf, VIMESemi

### VIME Self

- Create datamodule and model

In [19]:
dim = X_unlabeled.shape[1]

In [20]:
self_datamodule = VIMESelfDataModule(X_unlabeled, train_size=args_self.train_size, batch_size=args_self.batch_size, seed=args_self.seed)

In [21]:
vime_self = VIMESelf(
    in_features_list=[dim],
    out_features_list=[dim],
    learning_rate=args_self.learning_rate,
    p_masking=args_self.p_masking,
    alpha=args_self.alpha,
    log_interval=args_self.log_interval,
    seed=args_self.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


- Train vime self

In [22]:
checkpoint = ModelCheckpoint(
    dirpath=args_self.weights_dirpath,
    filename="vime_self",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode="min",
)

In [23]:
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_self.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(vime_self, self_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss: 0.2589 | Val Loss_m: 0.2196 | Val Loss_r: 0.0196
Epoch 1 | Train Loss: 0.4351 | Train Loss_m: 0.2388 | Train Loss_r: 0.0981  Val Loss: 0.2349 | Val Loss_m: 0.2037 | Val Loss_r: 0.0156
Epoch 5 | Train Loss: 0.2374 | Train Loss_m: 0.2064 | Train Loss_r: 0.0155  Val Loss: 0.2071 | Val Loss_m: 0.1762 | Val Loss_r: 0.0154
Epoch 10 | Train Loss: 0.2093 | Train Loss_m: 0.1784 | Train Loss_r: 0.0155  Val Loss: 0.1911 | Val Loss_m: 0.1606 | Val Loss_r: 0.0152
Epoch 15 | Train Loss: 0.1919 | Train Loss_m: 0.1614 | Train Loss_r: 0.0152  Val Loss: 0.1824 | Val Loss_m: 0.1524 | Val Loss_r: 0.0150
Epoch 20 | Train Loss: 0.1829 | Train Loss_m: 0.1527 | Train Loss_r: 0.0151  Val Loss: 0.1773 | Val Loss_m: 0.1478 | Val Loss_r: 0.0147
Epoch 25 | Train Loss: 0.1771 | Train Loss_m: 0.1475 | Train Loss_r: 0.0148  Val Loss: 0.1742 | Val Loss_m: 0.1444 | Val Loss_r: 0.0149
Epoch 30 | Train Loss: 0.1739 | Train Loss_m: 0.1443 | Train Loss_r: 0.0148  Val Loss: 0.1713 | Val Loss_m: 0.1421 | Val Loss_r

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


### VIME Semi

- Create datamodule and model

In [25]:
dim = X_labeled.shape[1]

In [26]:
semi_datamodule = VIMESemiDataModule(
    X_unlabeled,
    X_labeled,
    y,
    X_test,
    train_size=args_semi.train_size,
    labeled_batch_size=args_semi.labeled_batch_size,
    unlabeled_batch_size=args_semi.unlabeled_batch_size,
    seed=args_semi.seed,
)

In [27]:
best_model_path = checkpoint.best_model_path

In [28]:
vime_self_best = VIMESelf.load_from_checkpoint(best_model_path)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


In [29]:
pretrained_encoder = vime_self_best.encoder

In [30]:
vime_semi = VIMESemi(
    pretrained_encoder=pretrained_encoder,
    in_features_list=[dim, 128],
    out_features_list=[128, 128],
    num_classes=args_semi.num_classes,
    supervised_criterion=args_semi.supervised_criterion,
    learning_rate=args_semi.learning_rate,
    p_masking=args_semi.p_masking,
    K=args_semi.K,
    beta=args_semi.beta,
    log_interval=args_semi.log_interval,
    seed=args_semi.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


- Train vime semi

In [31]:
checkpoint = ModelCheckpoint(
    dirpath=args_semi.weights_dirpath,
    filename="vime_semi",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=20,
    mode="min",
)

In [32]:
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_semi.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [33]:
trainer.fit(vime_semi, semi_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss_s: 0.8024
Epoch 1 | Train Loss: 1.8722 | Train Loss_s: 1.4047 | Train Loss_u: 0.4675  Val Loss_s: 0.5521
Epoch 5 | Train Loss: 0.6824 | Train Loss_s: 0.4265 | Train Loss_u: 0.2559  Val Loss_s: 0.4939
Epoch 10 | Train Loss: 0.4860 | Train Loss_s: 0.2690 | Train Loss_u: 0.2170  Val Loss_s: 0.4651
Epoch 15 | Train Loss: 0.4274 | Train Loss_s: 0.2277 | Train Loss_u: 0.1998  Val Loss_s: 0.5001
Epoch 20 | Train Loss: 0.3492 | Train Loss_s: 0.1574 | Train Loss_u: 0.1917  Val Loss_s: 0.4266
Epoch 25 | Train Loss: 0.2984 | Train Loss_s: 0.1211 | Train Loss_u: 0.1773  Val Loss_s: 0.4370
Epoch 30 | Train Loss: 0.3204 | Train Loss_s: 0.1589 | Train Loss_u: 0.1614  Val Loss_s: 0.4717
Epoch 35 | Train Loss: 0.2912 | Train Loss_s: 0.1344 | Train Loss_u: 0.1568  Val Loss_s: 0.4541
Epoch 40 | Train Loss: 0.2802 | Train Loss_s: 0.1218 | Train Loss_u: 0.1584  Val Loss_s: 0.4224
Epoch 45 | Train Loss: 0.2790 | Train Loss_s: 0.1291 | Train Loss_u: 0.1499  Val Loss_s: 0.4567
Epoch 50 | Train Loss: 

In [34]:
pred = trainer.predict(vime_semi, semi_datamodule, ckpt_path=checkpoint.best_model_path)

INFO: Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt


In [35]:
pred = np.concatenate(pred).argmax(1)

In [36]:
accuracy_score(y_test, pred)

0.9114